## Agentic AI vs AI Agents
AI Agents
- "AI Agents" refers to individual software programs designed to perform specific tasks with a degree of autonomy (means without human intervetion)
- Functionality: AI agents handles specific tasks, limited autonomy
- Decision Making: follow predefined rules
- Example: customer service chatbot like automated banking bot which answer basic questions based on predefined knowledge like balance inquires, transactions

Agentic AI (LLM -> brain of Agentic AI)
- "Agentic AI" describes a broader framework where multiple AI agents can collaborate and make decisions independently to achieve a larger goal, emphasizing adaptability and learning from experience rather than just following predefined rules. (solve more complex workflows without human intervention)
- Functionality: Agentic AI focuses on the overall system with the ability to learn, adapt and make complex decision
- Decision Making: work on autonomous decision based on real time data and circumstances (make decison by own)
- Example: Smart home system having control of light, temperature, applicanes to have goal to reduce electricity, personalized health assistant system which analyze a patient's medical history, real-time health data, and even lifestyle factors to provide personalized care recommendations. Their ability to adapt to new mediacl research and patient feedback illustrates the higher flexibility and complexity of agentic AI
- Preception (gather the data from surrounding) -> Reasoning (Understand what is going on) -> Action (take a specific action) -> Learning (it improve and adapt over time)

## LangGraph
- LangGraph is a library for building **stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows**
- LangGraph is **DAG (Directed Acyclic Graph)** which follows sequential flow of information
- LangGraph is inspried by Pregel and Apache Beam. the public interface draws inspiration from NetworkX. 
- LangGraph is built by Langchain Inc, the creators of Langchain, but can be used without Langchain.
- LangGraph provides **fine-grained control over both the flow and state of your agent applications**
- It implements a central persistence layer, enabling features that are common to most agent architectures.
- **Memory:** LangGraph persists arbitrary aspects of your application's state, supporting memory of conversations and other updates within and across user interactions.
- **Human-in-the-loop:** because state is checkpointed, execution can be interrupted and resumed, allowing for decision, validation and corrections at key stages via human input.